In [95]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',50)

### Feature engineering
#### Credit reports

In [96]:
# Import a list of features associated with their data types
from data_credit_reports import *

In [97]:
credit_reports = pd.read_parquet('data/credit_reports.parquet')

##### New features
In this section I will create some new features based on the original ones

In [99]:
credit_reports['PERCENTAGE_OF_CREDIT_LIMIT_USED'] = \
    credit_reports['MAX_CREDIT'] / credit_reports['CREDIT_LIMIT']

In [100]:
credit_reports['PERCENTAGE_OF_CREDIT_LIMIT_USED'] = \
    credit_reports['PERCENTAGE_OF_CREDIT_LIMIT_USED'].replace(np.inf,1)

In [101]:
credit_reports[credit_reports['PERCENTAGE_OF_CREDIT_LIMIT_USED']>1][["MAX_CREDIT", "CREDIT_LIMIT"]]

,MAX_CREDIT,CREDIT_LIMIT
52,9696.0,7500.0
56,5467.0,4000.0
84,15000.0,12000.0
86,15000.0,12000.0
108,12897.0,6573.0
...,...,...
287275,8892.0,7500.0
287276,12784.0,10000.0
287277,1903.0,1605.0
287293,541.0,500.0


There are cases where the max credit used is greater than the actual credit limit, based on that I will limit PERCENTAGE_OF_CREDIT_LIMIT_USED to 1 and define a new variable

In [102]:
credit_reports['PERCENTAGE_OF_CREDIT_LIMIT_USED'] = \
    np.where(
        credit_reports['PERCENTAGE_OF_CREDIT_LIMIT_USED']>1, 1,
        credit_reports['PERCENTAGE_OF_CREDIT_LIMIT_USED']
        )

In [103]:
credit_reports['MAX_CREDIT_GREATER_THAN_CREDIT_LIMIT'] = \
    np.where(
        credit_reports['MAX_CREDIT'] > credit_reports['CREDIT_LIMIT'], 1, 0
        )

In [104]:
credit_reports['HAS_DELAYED_PAYMENTS'] = \
    np.where(
        credit_reports['DELAYED_PAYMENTS'] > 0, 1, 0
        )

In [105]:
credit_reports['HAS_WORST_DELAY'] = \
    np.where(
        credit_reports['WORST_DELAY'] > 0, 1, 0
        )

In [106]:
credit_reports['IS_CLOSED_LOAN'] = \
    np.where(
        credit_reports['LOAN_CLOSING_DATE'].isna(), 0, 1
        )

##### For clased loans  calculate the duration of the loan

In [107]:
# Covert datetime features to datetime
credit_reports[datetime_features] = credit_reports[datetime_features].apply(pd.to_datetime)

In [108]:
credit_reports['LOAN_DURATION'] = \
    (credit_reports['LOAN_CLOSING_DATE'] - credit_reports['LOAN_OPENING_DATE']).dt.days

##### For loans with the worst delay, calculate the time elapsed since the loan opening date to the date of the worst delay.

In [109]:
credit_reports['TIME_TO_WORST_DELAY'] = \
    (credit_reports['WORST_DELAY_DATE'] - credit_reports['LOAN_OPENING_DATE']).dt.days

###### Current payment
I'll process this variable as done in the EDA

In [110]:
credit_reports['CURRENT_PAYMENT'] = np.where(
    credit_reports['CURRENT_PAYMENT']=='V',-1,credit_reports['CURRENT_PAYMENT'])
credit_reports['CURRENT_PAYMENT'] = np.where(
    credit_reports['CURRENT_PAYMENT']=='--',-2,credit_reports['CURRENT_PAYMENT'])

#### Numerical features
Preprocess numerical features and create variables agregated at customer_id level

In [111]:
credit_reports[numerical_features] = credit_reports[numerical_features].apply(pd.to_numeric)

###### Minimun value of each feature

In [112]:
min_value_numerical_features = credit_reports.groupby('customer_id')[numerical_features].min()
min_value_numerical_features.columns = \
    list(map(lambda x: 'min_'+x,min_value_numerical_features.columns.tolist()))
min_value_numerical_features = min_value_numerical_features.reset_index()

In [113]:
min_value_numerical_features.head()

,customer_id,min_MAX_CREDIT,min_CREDIT_LIMIT,min_PAYMENT_AMOUNT,min_TOTAL_PAYMENTS,min_DELAYED_PAYMENTS,min_CURRENT_PAYMENT,min_WORST_DELAY,min_CURRENT_BALANCE,min_BALANCE_DUE,min_BALANCE_DUE_WORST_DELAY,min_PERCENTAGE_OF_CREDIT_LIMIT_USED,min_MAX_CREDIT_GREATER_THAN_CREDIT_LIMIT,min_HAS_DELAYED_PAYMENTS,min_HAS_WORST_DELAY,min_IS_CLOSED_LOAN,min_LOAN_DURATION,min_TIME_TO_WORST_DELAY
0,1,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.133750,0,0,0,0,336.0,NaN
1,2,1502.0,1.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.082527,0,0,0,0,372.0,NaN
2,3,10.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.083333,0,0,0,0,4.0,63.0
3,4,299.0,0.0,0.0,1.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.294118,0,0,0,0,5.0,99.0
4,5,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.007451,0,0,0,0,21.0,30.0


###### Maximum value of each feature

In [114]:
max_value_numerical_features = credit_reports.groupby('customer_id')[numerical_features].max()
max_value_numerical_features.columns = \
    list(map(lambda x: 'max_'+x,max_value_numerical_features.columns.tolist()))
max_value_numerical_features = max_value_numerical_features.reset_index()

In [115]:
max_value_numerical_features.head()

,customer_id,max_MAX_CREDIT,max_CREDIT_LIMIT,max_PAYMENT_AMOUNT,max_TOTAL_PAYMENTS,max_DELAYED_PAYMENTS,max_CURRENT_PAYMENT,max_WORST_DELAY,max_CURRENT_BALANCE,max_BALANCE_DUE,max_BALANCE_DUE_WORST_DELAY,max_PERCENTAGE_OF_CREDIT_LIMIT_USED,max_MAX_CREDIT_GREATER_THAN_CREDIT_LIMIT,max_HAS_DELAYED_PAYMENTS,max_HAS_WORST_DELAY,max_IS_CLOSED_LOAN,max_LOAN_DURATION,max_TIME_TO_WORST_DELAY
0,1,39291.0,161000.0,625.0,1.0,1.0,-1.0,1.0,12557.0,0.0,0.0,0.462313,0,1,1,1,2347.0,NaN
1,2,43712.0,91000.0,1140.0,37.0,0.0,-1.0,0.0,20429.0,0.0,0.0,1.000000,1,0,0,1,4542.0,NaN
2,3,441781.0,441781.0,4929.0,360.0,11.0,17.0,17.0,463885.0,4929.0,4929.0,1.000000,1,1,1,1,683.0,517.0
3,4,20046.0,18200.0,2607.0,43.0,1.0,2.0,2.0,19047.0,297.0,297.0,1.000000,1,1,1,1,63.0,111.0
4,5,218245.0,84406.0,84406.0,1000.0,54.0,84.0,84.0,84406.0,84406.0,68985.0,1.000000,1,1,1,1,1819.0,5594.0


###### Mean value of each feature

In [116]:
mean_value_numerical_features = credit_reports.groupby('customer_id')[numerical_features].mean()
mean_value_numerical_features.columns = \
    list(map(lambda x: 'mean_'+x,mean_value_numerical_features.columns.tolist()))
mean_value_numerical_features = mean_value_numerical_features.reset_index()

In [117]:
mean_value_numerical_features.head()

,customer_id,mean_MAX_CREDIT,mean_CREDIT_LIMIT,mean_PAYMENT_AMOUNT,mean_TOTAL_PAYMENTS,mean_DELAYED_PAYMENTS,mean_CURRENT_PAYMENT,mean_WORST_DELAY,mean_CURRENT_BALANCE,mean_BALANCE_DUE,mean_BALANCE_DUE_WORST_DELAY,mean_PERCENTAGE_OF_CREDIT_LIMIT_USED,mean_MAX_CREDIT_GREATER_THAN_CREDIT_LIMIT,mean_HAS_DELAYED_PAYMENTS,mean_HAS_WORST_DELAY,mean_IS_CLOSED_LOAN,mean_LOAN_DURATION,mean_TIME_TO_WORST_DELAY
0,1,21257.000000,69080.000000,202.625000,1.000000,0.125000,-1.000000,0.125000,2246.000000,0.000000,0.000000,0.331429,0.000000,0.125000,0.125000,0.250000,1341.500000,NaN
1,2,15227.500000,37566.666667,168.900000,18.500000,0.000000,-1.100000,0.000000,2577.900000,0.000000,0.000000,0.483688,0.100000,0.000000,0.000000,0.600000,1256.333333,NaN
2,3,14743.343750,16751.250000,379.468750,12.812500,2.656250,1.031250,1.718750,14773.718750,431.343750,431.343750,0.732001,0.093750,0.312500,0.312500,0.687500,67.090909,209.100000
3,4,3911.923077,2858.076923,421.307692,5.900000,0.153846,-0.615385,0.333333,3447.923077,41.076923,41.076923,0.833733,0.307692,0.153846,0.153846,0.461538,26.333333,105.000000
4,5,27515.775510,18409.478261,9184.857143,48.621622,6.204082,5.612245,9.326531,12025.612245,9401.102041,3056.836735,0.915384,0.530612,0.387755,0.551020,0.551020,528.000000,1062.785714


###### Join these numerical features

In [118]:
features_numeric = min_value_numerical_features.merge(
    max_value_numerical_features, on='customer_id')
features_numeric = features_numeric.merge(
    mean_value_numerical_features, on='customer_id')

In [119]:
features_numeric.head()

,customer_id,min_MAX_CREDIT,min_CREDIT_LIMIT,min_PAYMENT_AMOUNT,min_TOTAL_PAYMENTS,min_DELAYED_PAYMENTS,min_CURRENT_PAYMENT,min_WORST_DELAY,min_CURRENT_BALANCE,min_BALANCE_DUE,min_BALANCE_DUE_WORST_DELAY,min_PERCENTAGE_OF_CREDIT_LIMIT_USED,min_MAX_CREDIT_GREATER_THAN_CREDIT_LIMIT,min_HAS_DELAYED_PAYMENTS,min_HAS_WORST_DELAY,min_IS_CLOSED_LOAN,min_LOAN_DURATION,min_TIME_TO_WORST_DELAY,max_MAX_CREDIT,max_CREDIT_LIMIT,max_PAYMENT_AMOUNT,max_TOTAL_PAYMENTS,max_DELAYED_PAYMENTS,max_CURRENT_PAYMENT,max_WORST_DELAY,...,max_BALANCE_DUE_WORST_DELAY,max_PERCENTAGE_OF_CREDIT_LIMIT_USED,max_MAX_CREDIT_GREATER_THAN_CREDIT_LIMIT,max_HAS_DELAYED_PAYMENTS,max_HAS_WORST_DELAY,max_IS_CLOSED_LOAN,max_LOAN_DURATION,max_TIME_TO_WORST_DELAY,mean_MAX_CREDIT,mean_CREDIT_LIMIT,mean_PAYMENT_AMOUNT,mean_TOTAL_PAYMENTS,mean_DELAYED_PAYMENTS,mean_CURRENT_PAYMENT,mean_WORST_DELAY,mean_CURRENT_BALANCE,mean_BALANCE_DUE,mean_BALANCE_DUE_WORST_DELAY,mean_PERCENTAGE_OF_CREDIT_LIMIT_USED,mean_MAX_CREDIT_GREATER_THAN_CREDIT_LIMIT,mean_HAS_DELAYED_PAYMENTS,mean_HAS_WORST_DELAY,mean_IS_CLOSED_LOAN,mean_LOAN_DURATION,mean_TIME_TO_WORST_DELAY
0,1,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.133750,0,0,0,0,336.0,NaN,39291.0,161000.0,625.0,1.0,1.0,-1.0,1.0,...,0.0,0.462313,0,1,1,1,2347.0,NaN,21257.000000,69080.000000,202.625000,1.000000,0.125000,-1.000000,0.125000,2246.000000,0.000000,0.000000,0.331429,0.000000,0.125000,0.125000,0.250000,1341.500000,NaN
1,2,1502.0,1.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.082527,0,0,0,0,372.0,NaN,43712.0,91000.0,1140.0,37.0,0.0,-1.0,0.0,...,0.0,1.000000,1,0,0,1,4542.0,NaN,15227.500000,37566.666667,168.900000,18.500000,0.000000,-1.100000,0.000000,2577.900000,0.000000,0.000000,0.483688,0.100000,0.000000,0.000000,0.600000,1256.333333,NaN
2,3,10.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.083333,0,0,0,0,4.0,63.0,441781.0,441781.0,4929.0,360.0,11.0,17.0,17.0,...,4929.0,1.000000,1,1,1,1,683.0,517.0,14743.343750,16751.250000,379.468750,12.812500,2.656250,1.031250,1.718750,14773.718750,431.343750,431.343750,0.732001,0.093750,0.312500,0.312500,0.687500,67.090909,209.100000
3,4,299.0,0.0,0.0,1.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.294118,0,0,0,0,5.0,99.0,20046.0,18200.0,2607.0,43.0,1.0,2.0,2.0,...,297.0,1.000000,1,1,1,1,63.0,111.0,3911.923077,2858.076923,421.307692,5.900000,0.153846,-0.615385,0.333333,3447.923077,41.076923,41.076923,0.833733,0.307692,0.153846,0.153846,0.461538,26.333333,105.000000
4,5,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.007451,0,0,0,0,21.0,30.0,218245.0,84406.0,84406.0,1000.0,54.0,84.0,84.0,...,68985.0,1.000000,1,1,1,1,1819.0,5594.0,27515.775510,18409.478261,9184.857143,48.621622,6.204082,5.612245,9.326531,12025.612245,9401.102041,3056.836735,0.915384,0.530612,0.387755,0.551020,0.551020,528.000000,1062.785714


#### Categorical features
Preprocess categorical features and create variables agregated at customer_id level

In [120]:
credit_reports[categorical_features].head()

,PREVENTION_KEY,CURRENCY,PAYMENT_FREQUENCY,BUSINESS_TYPE,CREDIT_TYPE,ACCOUNT_TYPE,RESPONSABILITY_TYPE
0,SIN OBSERVACION,PESOS MEXICANOS,MENSUAL,TIENDA DEPARTAMENTAL,APARATOS/MUEBLES,PAGOS FIJOS,INDIVIDUAL (TITULAR)
1,SIN OBSERVACION,PESOS MEXICANOS,PAGO MINIMO PARA CUENTAS REVOLVENTES,TIENDA DEPARTAMENTAL,LINEA DE CREDITO,REVOLVENTE,INDIVIDUAL (TITULAR)
2,SIN OBSERVACION,PESOS MEXICANOS,SEMANAL,BANCOS,PRESTAMO PERSONAL,PAGOS FIJOS,INDIVIDUAL (TITULAR)
3,SIN OBSERVACION,PESOS MEXICANOS,SEMANAL,TIENDA DE ROPA,PRESTAMO PERSONAL,PAGOS FIJOS,INDIVIDUAL (TITULAR)
4,SIN OBSERVACION,PESOS MEXICANOS,PAGO MINIMO PARA CUENTAS REVOLVENTES,TIENDA DEPARTAMENTAL,LINEA DE CREDITO,REVOLVENTE,INDIVIDUAL (TITULAR)


In [121]:
for categorical_feature in categorical_features:
    # Obtain the percentage of each category in the variable
    frecuencies_categories = pd.DataFrame(
        credit_reports[categorical_feature].value_counts() / credit_reports.shape[0]
        ).reset_index()
    # Quit categories with less than 0.05% of frecuency
    frecuencies_categories = frecuencies_categories[
        frecuencies_categories[categorical_feature]>=0.005] 
    
    # Rename categories with less than 0.05% to 'OTROS'
    credit_reports[categorical_feature] = np.where(
        np.isin(
            credit_reports[categorical_feature].values, frecuencies_categories['index'].values
            )
        ,credit_reports[categorical_feature].values, 'OTROS'  
        )

###### In this section I will create dummy features related with categorical columns

In [126]:
categorical_columns = list()
for categorical_feature in categorical_features:
    categories = credit_reports[categorical_feature].unique()
    for category in categories:
        category_with_underscore = category.replace(' ', '_')
        categorical_column = categorical_feature + '_' + category_with_underscore
        categorical_columns.append(categorical_column)
        
        credit_reports[categorical_column] = np.where(
            credit_reports[categorical_feature]==category, 1, 0)

In [131]:
credit_reports[categorical_columns].head()

,PREVENTION_KEY_SIN_OBSERVACION,PREVENTION_KEY_CUENTA_CANCELADA_O_CERRADA,PREVENTION_KEY_OTROS,PREVENTION_KEY_CUENTA_EN_COBRANZA,PREVENTION_KEY_CUENTA_QUE_CAUSA_QUEBRANTO,PREVENTION_KEY_CUENTA_QUE_NO_ESTA_AL_CORRIENTE_VENDIDA_O_CEDIDA_A_UN_USUARIO_DE_UNA_SOCIEDAD,PREVENTION_KEY_CUENTA_INACTIVA,PREVENTION_KEY_CUENTA_CERRADA_QUE_ESTUVO_EN_COBRANZA_Y_FUE_PAGADA_SIN_CAUSAR_QUEBRANTO,CURRENCY_PESOS_MEXICANOS,CURRENCY_OTROS,PAYMENT_FREQUENCY_MENSUAL,PAYMENT_FREQUENCY_PAGO_MINIMO_PARA_CUENTAS_REVOLVENTES,PAYMENT_FREQUENCY_SEMANAL,PAYMENT_FREQUENCY_CATORCENAL,PAYMENT_FREQUENCY_QUINCENAL,PAYMENT_FREQUENCY_UNA_SOLA_EXHIBICION,PAYMENT_FREQUENCY_OTROS,BUSINESS_TYPE_TIENDA_DEPARTAMENTAL,BUSINESS_TYPE_BANCOS,BUSINESS_TYPE_OTROS,BUSINESS_TYPE_SERVICIO_DE_TELEVISION_DE_PAGA,BUSINESS_TYPE_MERCANCIA_PARA_HOGAR_Y_OFICINA,BUSINESS_TYPE_MICROFINANCIERA,BUSINESS_TYPE_BANCO,BUSINESS_TYPE_FINANCIERA,...,BUSINESS_TYPE_MERCANCIA_PARA_LA_CONSTRUCCION,BUSINESS_TYPE_FONDOS_Y_FIDEICOMISOS,BUSINESS_TYPE_GOBIERNO,CREDIT_TYPE_APARATOS/MUEBLES,CREDIT_TYPE_LINEA_DE_CREDITO,CREDIT_TYPE_PRESTAMO_PERSONAL,CREDIT_TYPE_OTROS,CREDIT_TYPE_CREDITO_AL_CONSUMO,CREDIT_TYPE_TARJETA_DE_CREDITO,CREDIT_TYPE_CREDITO_PERSONAL_AL_CONSUMO,CREDIT_TYPE_NO_DISPONIBLE,CREDIT_TYPE_GRUPO_SOLIDARIO,CREDIT_TYPE_PRESTAMO_DE_NOMINA,CREDIT_TYPE_CREDITO_FISCAL,ACCOUNT_TYPE_PAGOS_FIJOS,ACCOUNT_TYPE_REVOLVENTE,ACCOUNT_TYPE_SIN_LIMITE_PREESTABLECIDO,ACCOUNT_TYPE_POR_DETERMINAR,ACCOUNT_TYPE_OTROS,ACCOUNT_TYPE_HIPOTECA,RESPONSABILITY_TYPE_INDIVIDUAL_(TITULAR),RESPONSABILITY_TYPE_TITULAR_CON_AVAL,RESPONSABILITY_TYPE_OBLIGATORIO_SOLIDARIO,RESPONSABILITY_TYPE_OTROS,RESPONSABILITY_TYPE_MANCOMUNADO
0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0


###### Now I will aggregate these dummy features to customer_id level using the maximun of each feature in order to indicate the presence of that category in one of the loans and other with the sum to indicate how many loans have each category

In [132]:
max_value_categorical_features = credit_reports.groupby('customer_id')[categorical_columns].max()
max_value_categorical_features = max_value_categorical_features.reset_index()

In [134]:
max_value_categorical_features.head()

,customer_id,PREVENTION_KEY_SIN_OBSERVACION,PREVENTION_KEY_CUENTA_CANCELADA_O_CERRADA,PREVENTION_KEY_OTROS,PREVENTION_KEY_CUENTA_EN_COBRANZA,PREVENTION_KEY_CUENTA_QUE_CAUSA_QUEBRANTO,PREVENTION_KEY_CUENTA_QUE_NO_ESTA_AL_CORRIENTE_VENDIDA_O_CEDIDA_A_UN_USUARIO_DE_UNA_SOCIEDAD,PREVENTION_KEY_CUENTA_INACTIVA,PREVENTION_KEY_CUENTA_CERRADA_QUE_ESTUVO_EN_COBRANZA_Y_FUE_PAGADA_SIN_CAUSAR_QUEBRANTO,CURRENCY_PESOS_MEXICANOS,CURRENCY_OTROS,PAYMENT_FREQUENCY_MENSUAL,PAYMENT_FREQUENCY_PAGO_MINIMO_PARA_CUENTAS_REVOLVENTES,PAYMENT_FREQUENCY_SEMANAL,PAYMENT_FREQUENCY_CATORCENAL,PAYMENT_FREQUENCY_QUINCENAL,PAYMENT_FREQUENCY_UNA_SOLA_EXHIBICION,PAYMENT_FREQUENCY_OTROS,BUSINESS_TYPE_TIENDA_DEPARTAMENTAL,BUSINESS_TYPE_BANCOS,BUSINESS_TYPE_OTROS,BUSINESS_TYPE_SERVICIO_DE_TELEVISION_DE_PAGA,BUSINESS_TYPE_MERCANCIA_PARA_HOGAR_Y_OFICINA,BUSINESS_TYPE_MICROFINANCIERA,BUSINESS_TYPE_BANCO,...,BUSINESS_TYPE_MERCANCIA_PARA_LA_CONSTRUCCION,BUSINESS_TYPE_FONDOS_Y_FIDEICOMISOS,BUSINESS_TYPE_GOBIERNO,CREDIT_TYPE_APARATOS/MUEBLES,CREDIT_TYPE_LINEA_DE_CREDITO,CREDIT_TYPE_PRESTAMO_PERSONAL,CREDIT_TYPE_OTROS,CREDIT_TYPE_CREDITO_AL_CONSUMO,CREDIT_TYPE_TARJETA_DE_CREDITO,CREDIT_TYPE_CREDITO_PERSONAL_AL_CONSUMO,CREDIT_TYPE_NO_DISPONIBLE,CREDIT_TYPE_GRUPO_SOLIDARIO,CREDIT_TYPE_PRESTAMO_DE_NOMINA,CREDIT_TYPE_CREDITO_FISCAL,ACCOUNT_TYPE_PAGOS_FIJOS,ACCOUNT_TYPE_REVOLVENTE,ACCOUNT_TYPE_SIN_LIMITE_PREESTABLECIDO,ACCOUNT_TYPE_POR_DETERMINAR,ACCOUNT_TYPE_OTROS,ACCOUNT_TYPE_HIPOTECA,RESPONSABILITY_TYPE_INDIVIDUAL_(TITULAR),RESPONSABILITY_TYPE_TITULAR_CON_AVAL,RESPONSABILITY_TYPE_OBLIGATORIO_SOLIDARIO,RESPONSABILITY_TYPE_OTROS,RESPONSABILITY_TYPE_MANCOMUNADO
0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0
1,2,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0
2,3,1,1,1,0,0,1,0,0,1,0,1,0,1,0,1,1,0,0,0,1,0,1,1,1,...,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,0
3,4,1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0
4,5,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,0,1,...,0,0,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,0,0,0,1,0,0,0,0


In [135]:
sum_value_categorical_features = credit_reports.groupby('customer_id')[categorical_columns].sum()
sum_value_categorical_features.columns = \
    list(map(lambda x: 'sum_'+x,sum_value_categorical_features.columns.tolist()))
sum_value_categorical_features = sum_value_categorical_features.reset_index()

In [136]:
sum_value_categorical_features.head()

,customer_id,sum_PREVENTION_KEY_SIN_OBSERVACION,sum_PREVENTION_KEY_CUENTA_CANCELADA_O_CERRADA,sum_PREVENTION_KEY_OTROS,sum_PREVENTION_KEY_CUENTA_EN_COBRANZA,sum_PREVENTION_KEY_CUENTA_QUE_CAUSA_QUEBRANTO,sum_PREVENTION_KEY_CUENTA_QUE_NO_ESTA_AL_CORRIENTE_VENDIDA_O_CEDIDA_A_UN_USUARIO_DE_UNA_SOCIEDAD,sum_PREVENTION_KEY_CUENTA_INACTIVA,sum_PREVENTION_KEY_CUENTA_CERRADA_QUE_ESTUVO_EN_COBRANZA_Y_FUE_PAGADA_SIN_CAUSAR_QUEBRANTO,sum_CURRENCY_PESOS_MEXICANOS,sum_CURRENCY_OTROS,sum_PAYMENT_FREQUENCY_MENSUAL,sum_PAYMENT_FREQUENCY_PAGO_MINIMO_PARA_CUENTAS_REVOLVENTES,sum_PAYMENT_FREQUENCY_SEMANAL,sum_PAYMENT_FREQUENCY_CATORCENAL,sum_PAYMENT_FREQUENCY_QUINCENAL,sum_PAYMENT_FREQUENCY_UNA_SOLA_EXHIBICION,sum_PAYMENT_FREQUENCY_OTROS,sum_BUSINESS_TYPE_TIENDA_DEPARTAMENTAL,sum_BUSINESS_TYPE_BANCOS,sum_BUSINESS_TYPE_OTROS,sum_BUSINESS_TYPE_SERVICIO_DE_TELEVISION_DE_PAGA,sum_BUSINESS_TYPE_MERCANCIA_PARA_HOGAR_Y_OFICINA,sum_BUSINESS_TYPE_MICROFINANCIERA,sum_BUSINESS_TYPE_BANCO,...,sum_BUSINESS_TYPE_MERCANCIA_PARA_LA_CONSTRUCCION,sum_BUSINESS_TYPE_FONDOS_Y_FIDEICOMISOS,sum_BUSINESS_TYPE_GOBIERNO,sum_CREDIT_TYPE_APARATOS/MUEBLES,sum_CREDIT_TYPE_LINEA_DE_CREDITO,sum_CREDIT_TYPE_PRESTAMO_PERSONAL,sum_CREDIT_TYPE_OTROS,sum_CREDIT_TYPE_CREDITO_AL_CONSUMO,sum_CREDIT_TYPE_TARJETA_DE_CREDITO,sum_CREDIT_TYPE_CREDITO_PERSONAL_AL_CONSUMO,sum_CREDIT_TYPE_NO_DISPONIBLE,sum_CREDIT_TYPE_GRUPO_SOLIDARIO,sum_CREDIT_TYPE_PRESTAMO_DE_NOMINA,sum_CREDIT_TYPE_CREDITO_FISCAL,sum_ACCOUNT_TYPE_PAGOS_FIJOS,sum_ACCOUNT_TYPE_REVOLVENTE,sum_ACCOUNT_TYPE_SIN_LIMITE_PREESTABLECIDO,sum_ACCOUNT_TYPE_POR_DETERMINAR,sum_ACCOUNT_TYPE_OTROS,sum_ACCOUNT_TYPE_HIPOTECA,sum_RESPONSABILITY_TYPE_INDIVIDUAL_(TITULAR),sum_RESPONSABILITY_TYPE_TITULAR_CON_AVAL,sum_RESPONSABILITY_TYPE_OBLIGATORIO_SOLIDARIO,sum_RESPONSABILITY_TYPE_OTROS,sum_RESPONSABILITY_TYPE_MANCOMUNADO
0,1,6,2,0,0,0,0,0,0,8,0,8,0,0,0,0,0,0,0,0,0,1,0,0,5,...,0,0,0,0,2,0,1,0,5,0,0,0,0,0,0,6,2,0,0,0,8,0,0,0,0
1,2,5,3,2,0,0,0,0,0,10,0,10,0,0,0,0,0,0,0,0,2,0,0,0,7,...,0,0,0,0,0,0,1,0,9,0,0,0,0,0,1,9,0,0,0,0,10,0,0,0,0
2,3,24,6,1,0,0,1,0,0,32,0,28,0,2,0,1,1,0,0,0,1,0,2,17,1,...,0,0,0,0,0,19,1,11,0,0,0,0,1,0,29,0,0,2,0,1,32,0,0,0,0
3,4,12,1,0,0,0,0,0,0,13,0,11,0,0,0,2,0,0,0,0,0,0,0,2,4,...,0,0,0,0,0,6,0,2,3,0,0,0,2,0,10,3,0,0,0,0,13,0,0,0,0
4,5,26,8,4,2,6,2,1,0,49,0,27,0,4,0,10,6,2,7,1,3,2,4,0,13,...,0,0,6,5,6,5,2,1,17,2,0,0,5,6,20,18,11,0,0,0,49,0,0,0,0


###### Join these two set of features

In [137]:
features_categorical = max_value_categorical_features.merge(
    sum_value_categorical_features, on='customer_id')

In [138]:
features_categorical.head()

,customer_id,PREVENTION_KEY_SIN_OBSERVACION,PREVENTION_KEY_CUENTA_CANCELADA_O_CERRADA,PREVENTION_KEY_OTROS,PREVENTION_KEY_CUENTA_EN_COBRANZA,PREVENTION_KEY_CUENTA_QUE_CAUSA_QUEBRANTO,PREVENTION_KEY_CUENTA_QUE_NO_ESTA_AL_CORRIENTE_VENDIDA_O_CEDIDA_A_UN_USUARIO_DE_UNA_SOCIEDAD,PREVENTION_KEY_CUENTA_INACTIVA,PREVENTION_KEY_CUENTA_CERRADA_QUE_ESTUVO_EN_COBRANZA_Y_FUE_PAGADA_SIN_CAUSAR_QUEBRANTO,CURRENCY_PESOS_MEXICANOS,CURRENCY_OTROS,PAYMENT_FREQUENCY_MENSUAL,PAYMENT_FREQUENCY_PAGO_MINIMO_PARA_CUENTAS_REVOLVENTES,PAYMENT_FREQUENCY_SEMANAL,PAYMENT_FREQUENCY_CATORCENAL,PAYMENT_FREQUENCY_QUINCENAL,PAYMENT_FREQUENCY_UNA_SOLA_EXHIBICION,PAYMENT_FREQUENCY_OTROS,BUSINESS_TYPE_TIENDA_DEPARTAMENTAL,BUSINESS_TYPE_BANCOS,BUSINESS_TYPE_OTROS,BUSINESS_TYPE_SERVICIO_DE_TELEVISION_DE_PAGA,BUSINESS_TYPE_MERCANCIA_PARA_HOGAR_Y_OFICINA,BUSINESS_TYPE_MICROFINANCIERA,BUSINESS_TYPE_BANCO,...,sum_BUSINESS_TYPE_MERCANCIA_PARA_LA_CONSTRUCCION,sum_BUSINESS_TYPE_FONDOS_Y_FIDEICOMISOS,sum_BUSINESS_TYPE_GOBIERNO,sum_CREDIT_TYPE_APARATOS/MUEBLES,sum_CREDIT_TYPE_LINEA_DE_CREDITO,sum_CREDIT_TYPE_PRESTAMO_PERSONAL,sum_CREDIT_TYPE_OTROS,sum_CREDIT_TYPE_CREDITO_AL_CONSUMO,sum_CREDIT_TYPE_TARJETA_DE_CREDITO,sum_CREDIT_TYPE_CREDITO_PERSONAL_AL_CONSUMO,sum_CREDIT_TYPE_NO_DISPONIBLE,sum_CREDIT_TYPE_GRUPO_SOLIDARIO,sum_CREDIT_TYPE_PRESTAMO_DE_NOMINA,sum_CREDIT_TYPE_CREDITO_FISCAL,sum_ACCOUNT_TYPE_PAGOS_FIJOS,sum_ACCOUNT_TYPE_REVOLVENTE,sum_ACCOUNT_TYPE_SIN_LIMITE_PREESTABLECIDO,sum_ACCOUNT_TYPE_POR_DETERMINAR,sum_ACCOUNT_TYPE_OTROS,sum_ACCOUNT_TYPE_HIPOTECA,sum_RESPONSABILITY_TYPE_INDIVIDUAL_(TITULAR),sum_RESPONSABILITY_TYPE_TITULAR_CON_AVAL,sum_RESPONSABILITY_TYPE_OBLIGATORIO_SOLIDARIO,sum_RESPONSABILITY_TYPE_OTROS,sum_RESPONSABILITY_TYPE_MANCOMUNADO
0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,2,0,1,0,5,0,0,0,0,0,0,6,2,0,0,0,8,0,0,0,0
1,2,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,9,0,0,0,0,0,1,9,0,0,0,0,10,0,0,0,0
2,3,1,1,1,0,0,1,0,0,1,0,1,0,1,0,1,1,0,0,0,1,0,1,1,1,...,0,0,0,0,0,19,1,11,0,0,0,0,1,0,29,0,0,2,0,1,32,0,0,0,0
3,4,1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,6,0,2,3,0,0,0,2,0,10,3,0,0,0,0,13,0,0,0,0
4,5,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,0,1,...,0,0,6,5,6,5,2,1,17,2,0,0,5,6,20,18,11,0,0,0,49,0,0,0,0


###### And last but not least the number of loans of each customer

In [140]:
credit_reports['NUMBER_OF_LOANS'] = 1

In [146]:
number_of_loans = pd.DataFrame(credit_reports.groupby('customer_id')
                               ['NUMBER_OF_LOANS'].sum()).reset_index()

In [147]:
number_of_loans.head()

,customer_id,NUMBER_OF_LOANS
0,1,8
1,2,10
2,3,32
3,4,13
4,5,49


###### Combine all set of features 

In [148]:
features_credit_reports = features_numeric.merge(features_categorical, on='customer_id')
features_credit_reports = features_credit_reports.merge(number_of_loans, on='customer_id')

In [149]:
features_credit_reports.head()

,customer_id,min_MAX_CREDIT,min_CREDIT_LIMIT,min_PAYMENT_AMOUNT,min_TOTAL_PAYMENTS,min_DELAYED_PAYMENTS,min_CURRENT_PAYMENT,min_WORST_DELAY,min_CURRENT_BALANCE,min_BALANCE_DUE,min_BALANCE_DUE_WORST_DELAY,min_PERCENTAGE_OF_CREDIT_LIMIT_USED,min_MAX_CREDIT_GREATER_THAN_CREDIT_LIMIT,min_HAS_DELAYED_PAYMENTS,min_HAS_WORST_DELAY,min_IS_CLOSED_LOAN,min_LOAN_DURATION,min_TIME_TO_WORST_DELAY,max_MAX_CREDIT,max_CREDIT_LIMIT,max_PAYMENT_AMOUNT,max_TOTAL_PAYMENTS,max_DELAYED_PAYMENTS,max_CURRENT_PAYMENT,max_WORST_DELAY,...,sum_BUSINESS_TYPE_FONDOS_Y_FIDEICOMISOS,sum_BUSINESS_TYPE_GOBIERNO,sum_CREDIT_TYPE_APARATOS/MUEBLES,sum_CREDIT_TYPE_LINEA_DE_CREDITO,sum_CREDIT_TYPE_PRESTAMO_PERSONAL,sum_CREDIT_TYPE_OTROS,sum_CREDIT_TYPE_CREDITO_AL_CONSUMO,sum_CREDIT_TYPE_TARJETA_DE_CREDITO,sum_CREDIT_TYPE_CREDITO_PERSONAL_AL_CONSUMO,sum_CREDIT_TYPE_NO_DISPONIBLE,sum_CREDIT_TYPE_GRUPO_SOLIDARIO,sum_CREDIT_TYPE_PRESTAMO_DE_NOMINA,sum_CREDIT_TYPE_CREDITO_FISCAL,sum_ACCOUNT_TYPE_PAGOS_FIJOS,sum_ACCOUNT_TYPE_REVOLVENTE,sum_ACCOUNT_TYPE_SIN_LIMITE_PREESTABLECIDO,sum_ACCOUNT_TYPE_POR_DETERMINAR,sum_ACCOUNT_TYPE_OTROS,sum_ACCOUNT_TYPE_HIPOTECA,sum_RESPONSABILITY_TYPE_INDIVIDUAL_(TITULAR),sum_RESPONSABILITY_TYPE_TITULAR_CON_AVAL,sum_RESPONSABILITY_TYPE_OBLIGATORIO_SOLIDARIO,sum_RESPONSABILITY_TYPE_OTROS,sum_RESPONSABILITY_TYPE_MANCOMUNADO,NUMBER_OF_LOANS
0,1,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.133750,0,0,0,0,336.0,NaN,39291.0,161000.0,625.0,1.0,1.0,-1.0,1.0,...,0,0,0,2,0,1,0,5,0,0,0,0,0,0,6,2,0,0,0,8,0,0,0,0,8
1,2,1502.0,1.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.082527,0,0,0,0,372.0,NaN,43712.0,91000.0,1140.0,37.0,0.0,-1.0,0.0,...,0,0,0,0,0,1,0,9,0,0,0,0,0,1,9,0,0,0,0,10,0,0,0,0,10
2,3,10.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.083333,0,0,0,0,4.0,63.0,441781.0,441781.0,4929.0,360.0,11.0,17.0,17.0,...,0,0,0,0,19,1,11,0,0,0,0,1,0,29,0,0,2,0,1,32,0,0,0,0,32
3,4,299.0,0.0,0.0,1.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.294118,0,0,0,0,5.0,99.0,20046.0,18200.0,2607.0,43.0,1.0,2.0,2.0,...,0,0,0,0,6,0,2,3,0,0,0,2,0,10,3,0,0,0,0,13,0,0,0,0,13
4,5,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.007451,0,0,0,0,21.0,30.0,218245.0,84406.0,84406.0,1000.0,54.0,84.0,84.0,...,0,6,5,6,5,2,1,17,2,0,0,5,6,20,18,11,0,0,0,49,0,0,0,0,49


##### Add these credit reports features to the main dataset

In [150]:
main_dataset = pd.read_csv('data/main_dataset_preprocessed.csv')

In [152]:
main_dataset.shape

(12441, 25)

In [155]:
main_dataset = main_dataset.merge(features_credit_reports, on='customer_id',how='left')

In [161]:
main_dataset.head()

,customer_id,loan_id,ACC_CREATION_DATETIME,APPLICATION_DATETIME,LOAN_ORIGINATION_DATETIME,max_days_late,target,account_to_application_days,n_sf_apps,first_app_date,last_app_date,n_bnpl_apps,n_bnpl_approved_apps,first_bnpl_app_date,last_bnpl_app_date,n_inquiries_l3m,n_inquiries_l6m,antiquity_account,last_app_date_to_application_days,first_app_date_to_application_days,last_bnpl_app_date_to_application_days,first_bnpl_app_date_to_application_days,first_app_date_to_last_app_date_days,first_bnpl_app_date_to_last_bnpl_app_date_days,vintage,...,sum_BUSINESS_TYPE_FONDOS_Y_FIDEICOMISOS,sum_BUSINESS_TYPE_GOBIERNO,sum_CREDIT_TYPE_APARATOS/MUEBLES,sum_CREDIT_TYPE_LINEA_DE_CREDITO,sum_CREDIT_TYPE_PRESTAMO_PERSONAL,sum_CREDIT_TYPE_OTROS,sum_CREDIT_TYPE_CREDITO_AL_CONSUMO,sum_CREDIT_TYPE_TARJETA_DE_CREDITO,sum_CREDIT_TYPE_CREDITO_PERSONAL_AL_CONSUMO,sum_CREDIT_TYPE_NO_DISPONIBLE,sum_CREDIT_TYPE_GRUPO_SOLIDARIO,sum_CREDIT_TYPE_PRESTAMO_DE_NOMINA,sum_CREDIT_TYPE_CREDITO_FISCAL,sum_ACCOUNT_TYPE_PAGOS_FIJOS,sum_ACCOUNT_TYPE_REVOLVENTE,sum_ACCOUNT_TYPE_SIN_LIMITE_PREESTABLECIDO,sum_ACCOUNT_TYPE_POR_DETERMINAR,sum_ACCOUNT_TYPE_OTROS,sum_ACCOUNT_TYPE_HIPOTECA,sum_RESPONSABILITY_TYPE_INDIVIDUAL_(TITULAR),sum_RESPONSABILITY_TYPE_TITULAR_CON_AVAL,sum_RESPONSABILITY_TYPE_OBLIGATORIO_SOLIDARIO,sum_RESPONSABILITY_TYPE_OTROS,sum_RESPONSABILITY_TYPE_MANCOMUNADO,NUMBER_OF_LOANS
0,1223,1,2021-08-23 13:57:56.000,2022-04-26 07:00:00.000,2022-10-20 18:26:35.000,5,0,245,1.0,2022-04-25 00:00:00.000,2022-04-25 00:00:00.000,1.0,0.0,2022-04-26 02:15:17.742,2022-04-26 02:15:17.742,91.0,91.0,245,1.0,1.0,0.0,0.0,0.0,0.0,2022-10,...,0.0,0.0,5.0,7.0,20.0,2.0,13.0,4.0,3.0,0.0,0.0,0.0,0.0,43.0,6.0,5.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,0.0,54.0
1,5190,2,2022-04-26 09:57:25.000,2022-04-26 07:00:00.000,2022-12-22 18:03:32.000,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2022-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5194,3,2022-04-26 12:22:35.000,2022-04-26 07:00:00.000,2022-08-09 17:31:05.000,4,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2022-8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4535,5,2022-04-01 14:28:42.000,2022-04-26 07:00:00.000,2022-07-24 12:07:15.000,2,0,24,1.0,2022-04-01 00:00:00.000,2022-04-01 00:00:00.000,1.0,0.0,2022-04-01 19:37:56.185,2022-04-01 19:37:56.185,21.0,21.0,24,25.0,25.0,24.0,24.0,0.0,0.0,2022-7,...,0.0,0.0,0.0,0.0,8.0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,2.0,0.0,0.0,1.0,14.0,0.0,0.0,0.0,0.0,14.0
4,3604,6,2022-02-21 11:55:32.000,2022-05-05 07:00:00.000,2022-12-08 07:12:18.000,7,0,72,1.0,2022-02-21 00:00:00.000,2022-02-21 00:00:00.000,NaN,NaN,NaN,NaN,26.0,26.0,72,73.0,73.0,NaN,NaN,0.0,NaN,2022-12,...,0.0,0.0,0.0,1.0,19.0,3.0,6.0,3.0,0.0,0.0,0.0,1.0,0.0,26.0,4.0,2.0,0.0,0.0,1.0,33.0,0.0,0.0,0.0,0.0,33.0


In [159]:
main_dataset.to_csv('data/features_preprocessed.csv', index=False)